In [132]:
from d2_net import Deception_Net
import numpy as np
from keras.utils import to_categorical
from preprocess import get_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preprocess Data

In [133]:
data, labels = get_data()

In [134]:
p = np.random.permutation(len(labels))
labels = labels[p]
data = data[p]

train_data = data[:135]
train_labels = labels[:135]
test_data = data[135:]
test_labels = labels[135:]

In [135]:
expanded_train_labels = np.empty((train_labels.shape[0] * 60))
for i in range(train_labels.shape[0]):
    expanded_train_labels[i*60:(i+1)*60] = np.repeat(train_labels[i], 60)
expanded_train_labels = to_categorical(expanded_train_labels)

In [136]:
expanded_test_labels = np.empty((test_labels.shape[0] * 60))
for i in range(test_labels.shape[0]):
    expanded_test_labels[i*60:(i+1)*60] = np.repeat(test_labels[i], 60)
expanded_test_labels_cat = to_categorical(expanded_test_labels)

In [137]:
train_data_frames = np.reshape(train_data, newshape=(-1,250,80,1))
test_data_frames = np.reshape(test_data, newshape=(-1,250,80,1))

In [138]:
expanded_train_labels.shape

(8100, 2)

# Model

In [139]:
model = Deception_Net(weights_path="bball.h5")
model.model.fit(train_data_frames, expanded_train_labels, validation_data=(test_data_frames, expanded_test_labels_cat), epochs=3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 248, 78, 32)       320       
_________________________________________________________________
max_pooling2d_65 (MaxPooling (None, 124, 39, 32)       0         
_________________________________________________________________
batch_normalization_65 (Batc (None, 124, 39, 32)       128       
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 122, 37, 64)       18496     
_________________________________________________________________
max_pooling2d_66 (MaxPooling (None, 61, 18, 64)        0         
_________________________________________________________________
batch_normalization_66 (Batc (None, 61, 18, 64)        256       
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 59, 16, 64)        36928     
__________

In [142]:
from PIL import Image

img = Image.fromarray(np.squeeze(data[1][59]), 'L')
img.show()

In [143]:
preds = model.predict(test_data)
predict = np.argmax(preds, axis=1)

In [144]:
unique, counts = np.unique(np.equal(expanded_test_labels,predict), return_counts=True)
dict(zip(unique, counts)) 

{False: 529, True: 371}

In [146]:
vid_predict = np.reshape(predict, newshape=(15,60))
predictions = np.empty(15,)
for idx, vid in enumerate(vid_predict):
    predictions[idx] = np.argmax(np.bincount(vid))
print("predictions", predictions)
print(" and labels", [int(x) for x in test_labels])

predictions [1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 and labels [0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0]


# Saliency

In [150]:
from vis.visualization import visualize_saliency
from vis.utils import utils
from keras import activations

model.model.layers[15].activation = activations.linear
model.model = utils.apply_modifications(model.model)

missed = test_data[np.where(test_labels == 0)[0]]
missed_frames = np.reshape(missed, newshape=(-1,250,80,1))
grads = np.empty(shape=(missed_frames.shape[0], 250, 80))
for i in range(missed_frames.shape[0]):
    print(i)
    grads[i] = visualize_saliency(model.model, layer_idx=15, filter_indices=0, seed_input=missed_frames[i], backprop_modifier='guided')

0
1
2
3
4
5
6
7
8
9


In [154]:
test_data.shape

(900, 250, 80, 1)

In [151]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

numFrames = missed_frames.shape[0]
fig, ax = plt.subplots(1,2)
im1 = ax[0].imshow(grads[0], cmap="jet")
im2 = ax[1].imshow(np.squeeze(missed_frames[0]))

def update(i):
    im1.set_data(grads[i])
    im2.set_data(np.squeeze(missed_frames[i]))
    return fig

ani = animation.FuncAnimation(fig, update, frames=numFrames, repeat=False)
display(HTML(ani.to_html5_video()))